# Altairで「マンガ雑誌のデータ」の図を作り直す

今回は、文化庁の[メディア芸術データベース・ラボ（MADB Lab）](https://mediag.bunka.go.jp/madb_lab/)で公開されている四大少年誌（週刊少年サンデー、週刊少年ジャンプ、週刊少年チャンピオン、週刊少年マガジン）のデータを使って、量的データと質的データの可視化を練習します。

まず、「四大少年誌それぞれの掲載作品のジャンルと著者にはどのような特徴があるのか？」という大きな問いを立て、可視化手法を学びながらデータを見て、具体的な問いを決めていきましょう。

[マンガと学ぶデータビジュアライゼーション](https://kakeami.github.io/viz-madb/index.html)の内容を全面的に参考にし、一部改変しています。

[Plotlyで作られた図](./visualizing-quantitative-and-qualitative-data.ipynb)の中から、好きな図を3つ選んで、Altairで再現しましょう。

再現する時には、変数の性質（名義、順序、間隔、比例）に注意して、どのタイプの図がどの変数をうまく扱えるかを考えてみましょう。

複雑な図を再現したい時は、[Altair Example Gallery](https://altair-viz.github.io/gallery/index.html)を参考にしてみてください。

## ライブラリの読み込み

In [ ]:
# ライブラリのインストール。必要に応じてコメントアウトする。
# !pip install altair
# !pip install vl-convert-python

In [ ]:
import pandas as pd

import altair as alt

In [ ]:
import itertools
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 5000行の制限をはずす
alt.data_transformers.disable_max_rows()

## 準備関数

In [ ]:
# weekdayを曜日に変換
WD2STR = {
    0: 'Mon.',
    1: 'Tue.',
    2: 'Wed.',
    3: 'Thu.',
    4: 'Fri.',
    5: 'Sat.',
    6: 'Sun.',}

In [ ]:
def add_years_to_df(df, unit_years=10):
    """unit_years単位で区切ったyears列を追加"""
    df_new = df.copy()
    df_new['years'] = \
        pd.to_datetime(df['datePublished']).dt.year \
        // unit_years * unit_years
    df_new['years'] = df_new['years'].astype(str)
    return df_new

In [ ]:
def add_weekday_to_df(df):
    """曜日情報をdfに追加"""
    df_new = df.copy()
    df_new['weekday'] = \
        pd.to_datetime(df_new['datePublished']).dt.weekday
    df_new['weekday_str'] = df_new['weekday'].apply(
        lambda x: WD2STR[x])
    return df_new

In [ ]:
def add_mcid_to_df(df):
    """mcnameのindexをdfに追加"""
    df_new = df.copy()
    mcname2mcid = {
        x: i for i, x in enumerate(df['mcname'].unique())}
    df_new['mcid'] = df_new['mcname'].apply(
        lambda x: mcname2mcid[x])
    return df_new

In [ ]:
def resample_df_by_cname_and_years(df):
    """cnameとyearsのすべての組み合わせが存在するように0埋め
    この処理を実施しないと作図時にX軸方向の順序が変わってしまう"""
    df_new = df.copy()
    yearss = df['years'].unique()
    cnames = df['cname'].unique()
    for cname, years in itertools.product(cnames, yearss):
        df_tmp = df_new[
            (df_new['cname'] == cname)&\
            (df_new['years'] == years)]
        if df_tmp.shape[0] == 0:
            s = pd.DataFrame(
                {'cname': cname,
                 'years': years,
                 'weeks': 0,},
                index=df_tmp.columns)
            df_new = pd.concat(
                [df_new, s], ignore_index=True)
    return df_new

In [ ]:
def resample_df_by_creator_and_years(df):
    """creatorとyearsのすべての組み合わせが存在するように0埋め
    この処理を実施しないと作図時にX軸方向の順序が変わってしまう"""
    df_new = df.copy()
    yearss = df['years'].unique()
    creators = df['creator'].unique()
    for creator, years in itertools.product(creators, yearss):
        df_tmp = df_new[
            (df_new['creator'] == creator)&\
            (df_new['years'] == years)]
        if df_tmp.shape[0] == 0:
            s = pd.Series(
                {'creator': creator,
                 'years': years,
                 'weeks': 0,},
                index=df_tmp.columns)
            df_new = pd.concat(
                [df_new, s], ignore_index=True)
    return df_new

## データの用意

四大少年誌の`1970-07-27`から`2017-07-06`までの全ての掲載作品のデータを使います。

すでに前処理がされているデータがあるので、そちらを使います。

In [ ]:
file = "./data/episodes.csv"

In [ ]:
df = pd.read_csv(file)

In [ ]:
df.shape

各週の掲載作品を一行ずつ格納しているため、合計で約18万行程度になります。

In [ ]:
df.columns

- `mcname`: 雑誌名（**M**gazine **C**ollection **NAME**）
- `miid`：雑誌巻号ID（**M**agazine **I**tem **ID**）
- `miname`: 雑誌巻号名（**M**agazine **I**tem **NAME**）
- `cid`: マンガ作品ID（**C**omic **ID**）
- `cname`: マンガ作品名（**C**omic **NAME**）
- `epname`: 各話タイトル（**EP**isode **NAME**）
- `creator`: 作者名
- `pageStart`: 開始ページ
- `pageEnd`: 終了ページ
- `numberOfPages`: 雑誌の合計ページ数
- `datePublished`: 雑誌の発行日
- `price`: 雑誌の価格
- `publisher`: 雑誌の出版社
- `editor`: 雑誌の編集者（編集長）
- `pages`: 各話のページ数（`pageEnd` - `pageStart` + 1）
- `pageEndMax`: 雑誌に掲載されているマンガ作品のうち，`pageEnd`の最大値
- `pageStartPosition`: 各話の`pageStart`の相対的な位置（`pageStart` / `pageEndMax`）

In [ ]:
df.head()

In [ ]:
df.describe()

欠損値を確認してみます。

特に`epname`と`publisher`の欠測が多いことがわかります．

In [ ]:
df.isna().sum().reset_index()

## デザインガイド

* 図の構成要素
    * 全体のサイズ：650〜900px
    * 図番号：つけない
    * タイトル：上部につける
    * サブタイトル：タイトル下部につける
    * 軸タイトル：つける
    * 軸ラベル：つける
    * 軸と目盛り：つける
    * グリッド線：横方向のみつける
    * データソース・注記：下部につける
    * ロゴ：つけない
    * 凡例：必要に応じてつける
    * ラベル：必要に応じてつける
* 色
    * データに基づいてカラースケールを決める
    * CVDセーフな配色を使う
* 書体
    * キャプション：源ノ角ゴシック
    * ラベル：Input Mono Condensed
* 保存形式：PDF
* アクセシビリティ：
    * 文字を大きくする
    * 行間を開ける
    * CVDセーフな配色を使う

In [ ]:
ud_colors = ["#e69f00", "#56b4e9", "#009e73", "#f0e442", "#0072b2", "#d55e00", "#cc79a7"]

## （例）Altairで再現1：作品別・年代別の掲載週数（上位20作品）

In [ ]:
# dfに10年区切りの年代情報を追加
df = add_years_to_df(df)

In [ ]:
# プロット用に集計
df_plot = df.groupby('cname')['years'].value_counts().\
    reset_index(name='weeks')
# 連載週数上位10作品を抽出
cnames = list(df.value_counts('cname').head(20).index)
df_plot = df_plot[df_plot['cname'].isin(cnames)].\
    reset_index(drop=True)
# cname，yearsでアップサンプリング
df_plot = resample_df_by_cname_and_years(df_plot)

In [ ]:
# 合計連載週数で降順ソート
df_plot['order'] = df_plot['cname'].apply(
    lambda x: cnames.index(x))
df_plot = df_plot.sort_values(
    ['order', 'years'], ignore_index=True)

In [ ]:
# 自分のシステムに応じてフォント名を書き換えてください

## macOS
## 以下のフォントは下記URLからダウンロードできます
## IBM Plex Mono: https://fonts.google.com/specimen/IBM+Plex+Mono
## Noto Sans Japanese: https://fonts.google.com/noto/specimen/Noto+Sans+JP
## お好みで自分のシステムに入っているフォントを使ってください
## フォント名はFont Book.appで確認できます
## 新しくフォントをインストールした後は、VS Codeを再起動してください
label_font = "IBM Plex Mono"
caption_font = "Noto Sans JP"

## Ubuntu/WSL
## sudo apt install fonts-noto-cjk
## 上記コマンドでフォントをインストールし、VS Codeを再起動してください
## fc-list
## でインストール済みのフォントを確認することができます
# label_font = "Ubuntu Sans Mono"
# caption_font = "Noto Sans CJK JP"

## Windows (Git Bash)
## 「コントロールパネル > デスクトップのカスタマイズ > フォント」からフォント名を確認することができます
## 新しくフォントをインストールした後は、VS Codeを再起動してください
# label_font = "Consolas"
# caption_font = "BIZ UDゴシック"

## 参考：https://infovis.zhuxinru.com/misc/fonts

In [ ]:
chart = alt.Chart(df_plot).mark_bar().encode(
    alt.X("cname:N", sort="-y", title="作品名", axis=alt.Axis(grid=False, labelPadding=10, labelLimit=200)),
    alt.Y("weeks:Q", title="掲載週数", axis=alt.Axis(grid=True, titleX=-50, labelFont=label_font)),
    alt.XOffset("years:N"),
    alt.Color("years:N", title="年代", scale=alt.Scale(range=ud_colors)),
    alt.Tooltip(["weeks"])
).properties(
    title="作品別・年代別の掲載週数（上位20作品）",
    width=900
).configure_axis(
    labelFont=caption_font,
    labelFontSize=14,
    titleFont=caption_font,
    titleFontSize=16
).configure_title(
    font=caption_font,
    fontSize=18
).configure_legend(
    labelFont=label_font,
    labelFontSize=14,
    titleFont=caption_font,
    titleFontSize=16
)

chart.save("./figs/weeks_by_cname_and_years.pdf")
chart.save("./figs/weeks_by_cname_and_years.png", scale_factor=1.0)

In [ ]:
chart

## （例）Altairで再現2：雑誌別の年毎のエピソード数の推移

In [ ]:
# 1年単位で区切ったyearsを追加
df = add_years_to_df(df, 1)

In [ ]:
df_plot = df.groupby(["years", "mcname"])["cname"].count().reset_index(name="count")

In [ ]:
chart = alt.Chart(df_plot).mark_line().encode(
    alt.X("years:O", title="年代", axis=alt.Axis(grid=False)),
    alt.Y("count:Q", title="作品数", axis=alt.Axis(grid=True, titleX=-50)),
    alt.Color("mcname:N", title="雑誌名", scale=alt.Scale(range=ud_colors)),
).properties(
    title="雑誌別の年毎のエピソード数の推移",
    width=800
).configure_axis(
    labelFont=label_font,
    labelFontSize=14,
    titleFont=caption_font,
    titleFontSize=16
).configure_title(
    font=caption_font,
    fontSize=18
).configure_legend(
    labelFont=caption_font,
    labelFontSize=14,
    titleFont=caption_font,
    titleFontSize=16
)

chart.save("./figs/episodes_by_years_and_mcname.pdf")

In [ ]:
chart

---

この演習は以上です。